In [1]:

import pandas as pd
import os

In [2]:
#Download the initial test statistics
df = pd.read_csv('Data/permutation_test_files_proteomics/initial_tstats.csv', index_col=0)


In [3]:

path = os.getcwd() + '\\Data\\permutation_test_files_proteomics'

In [3]:
edgelist = df.index
edgelist[0]

'R-HSA-109606, R-HSA-109581'

In [4]:
pathway_list = []
sig_edges = []

for index,pathway_pair in enumerate(edgelist[:5]):   #test all pathways
    for filename in os.listdir(path):
        if filename != 'initial_tstats.csv':
            with open(os.path.join(path, filename)) as file_in:    
                lines = file_in.readlines()
                pathway_stat = float(lines[0].split(',')[index])
                pathway_list.append(pathway_stat)
    
    print(pathway_list, pathway_pair)

    comparison = df.Initial_tstat[index]    #get initial test statistic
    print(comparison)
    counter = 0
    
    for num in pathway_list:  #len(pathway_list) = number of permutations
        if abs(num) >= comparison:   
            counter += 1
    
    print(counter)
    p_val = (counter/len(pathway_list))    #divide number of tests above or equal to the test statistic by total number of tests

    if p_val < 0.01:  #for 100,000 permutations
        sig_edges.append(pathway_pair)  

    pathway_list = []

print(sig_edges)

NameError: name 'path' is not defined

In [26]:
sig_edges

[]

In [35]:
with open("Data/permutation_test_files_proteomics/Run9.txt") as file_in:    
                lines = file_in.readlines()
                pathway_stat = float(lines[0].split(',')[1])
                print(pathway_stat)

-0.5646745805638236


In [ ]:
with open ('Data/permutation_test_files_proteomics/sig_edges.txt', 'w') as file:
     file.write(','.join(str(i) for i in sig_edges))